In [ ]:
# !pip install geemap
# !pip install earthengine-api


In [ ]:
import geemap
import ee
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
ee.Authenticate()
# Initialize the Earth Engine module.
ee.Initialize()

In [ ]:
# Define the location (latitude, longitude)
occurrence_point = ee.Geometry.Point([longitude, latitude])

# Load Landsat 8 Image Collection and filter by date and location
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(occurrence_point) \
    .filterDate('2020-01-01', '2020-12-31') \
    .sort('CLOUD_COVER')

# Select the first image with the least cloud cover
landsat_image = landsat_collection.first()

# Select the bands of interest (excluding thermal and quality bands)
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
landsat_bands = landsat_image.select(bands)

# Sample the image at the occurrence point
sample = landsat_bands.sample(region=occurrence_point, scale=30).first().toDictionary().getInfo()

# Convert the sample to a numpy array
values = np.array(list(sample.values())).reshape(1, -1)

# Perform PCA
pca = PCA(n_components=3)
pca_result = pca.fit_transform(values)

# Plot the first two PCA components
plt.scatter(pca_result[:, 0], pca_result[:, 1])
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA of Landsat Bands')
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression
import folium

# Assuming you have labeled data with occurrence points (1 for presence, 0 for absence)
# For demonstration, we will generate some synthetic data
num_samples = 100
X = np.random.rand(num_samples, 3)  # Random features for PCA components
y = np.random.randint(2, size=num_samples)  # Random binary labels

# Train the logistic regression model
log_reg = LogisticRegression()
log_reg.fit(X, y)

# Predict for a grid of points to generate the predictive map
latitudes = np.linspace(min_lat, max_lat, 100)
longitudes = np.linspace(min_lon, max_lon, 100)

predictions = []

for lat in latitudes:
    for lon in longitudes:
        point = ee.Geometry.Point([lon, lat])
        sample = landsat_bands.sample(region=point, scale=30).first().toDictionary().getInfo()
        values = np.array(list(sample.values())).reshape(1, -1)
        pca_result = pca.transform(values)
        pred = log_reg.predict(pca_result)
        predictions.append((lat, lon, pred[0]))

# Create a map to visualize the predictions
m = folium.Map(location=[latitude, longitude], zoom_start=8)

for lat, lon, pred in predictions:
    color = 'green' if pred == 1 else 'red'
    folium.CircleMarker(location=[lat, lon], radius=2, color=color).add_to(m)

# Save the map
m.save('predictive_map.html')


In [ ]:
import geemap
import ee
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import folium

# Initialize the Earth Engine module.
ee.Initialize()

# Define the location (latitude, longitude)
longitude, latitude = -122.292, 37.827  # example coordinates
occurrence_point = ee.Geometry.Point([longitude, latitude])

# Load Landsat 8 Image Collection and filter by date and location
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(occurrence_point) \
    .filterDate('2020-01-01', '2020-12-31') \
    .sort('CLOUD_COVER')

# Select the first image with the least cloud cover
landsat_image = landsat_collection.first()

# Select the bands of interest (excluding thermal and quality bands)
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
landsat_bands = landsat_image.select(bands)

# Sample the image at the occurrence point
sample = landsat_bands.sample(region=occurrence_point, scale=30).first().toDictionary().getInfo()

# Convert the sample to a numpy array
values = np.array(list(sample.values())).reshape(1, -1)

# Perform PCA
pca = PCA(n_components=3)
pca_result = pca.fit_transform(values)

# Plot the first two PCA components
plt.scatter(pca_result[:, 0], pca_result[:, 1])
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA of Landsat Bands')
plt.show()
